In [ ]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
    output_df = input_df.withColumn("ingestion_date", current_timestamp())
    return output_df

In [ ]:
from delta.tables import DeltaTable

def merge_delta_lake(table_name, table_path, merge_condition, input_df, partition_column):

    if spark._jsparkSession.catalog().tableExists(table_name):
        delta_table = DeltaTable.forPath(spark, table_path)

        (delta_table.alias("tgt")
            .merge(
                input_df.alias("src"),
                merge_condition
            )
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
    else:
        (input_df.write
            .format("delta")
            .mode("overwrite")
            .partitionBy(partition_column)
            .saveAsTable(table_name)
        )